In [65]:
import psycopg2
#建立数据库连接
con = psycopg2.connect(database="zxzj", user="postgres", password="123456", host="localhost", port="5432")
print("connection successful!\n")

connection successful!


In [66]:
autocommit = psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
con.set_isolation_level(autocommit)
cur = (con.cursor())

In [7]:
create_bookDB_sql = """
CREATE TABLE IF NOT EXISTS bookDB (
    id TEXT primary key,
    title TEXT,
    author TEXT,
    publisher TEXT,
    original_title TEXT,
    translator TEXT,
    pub_year TEXT,
    pages INTEGER,
    price INTEGER,
    currency_unit TEXT,
    binding TEXT,
    isbn TEXT,
    author_intro TEXT,
    book_intro text,
    content TEXT,
    tags TEXT
    );
"""
cur.execute(create_bookDB_sql)

In [12]:
import sqlite3
import os

os.chmod('D:/bookstore2/bookstoreTask/CDMS.Xuan_ZHOU.2024Fall.DaSE/project1/bookstore/fe/data/book_lx.db', 0o664)  
conn = sqlite3.connect('D:/bookstore2/bookstoreTask/CDMS.Xuan_ZHOU.2024Fall.DaSE/project1/bookstore/fe/data/book_lx.db')  # 确保路径正确
cursor = conn.cursor()
cursor.execute("SELECT * FROM book")
rows = cursor.fetchall()

insert_query = """
INSERT INTO bookDB (id, title, author, publisher, original_title, translator, pub_year, pages, price, 
                     currency_unit, binding, isbn, author_intro, book_intro, content, tags)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT(id) DO NOTHING;
"""

# 将从 SQLite 获取的每一行数据插入到 PostgreSQL 表中
for row in rows:
    book_data = (
        row[0],  # id
        row[1],  # title
        row[2],  # author
        row[3],  # publisher
        row[4],  # original_title
        row[5],  # translator
        row[6],  # pub_year
        row[7],  # pages
        row[8],  # price
        row[9],  # currency_unit
        row[10], # binding
        row[11], # isbn
        row[12], # author_intro
        row[13], # book_intro
        row[14], # content
        row[15], # tags
    )
    
    cur.execute(insert_query, book_data)

In [13]:
con.commit()
conn.close()

In [14]:
sql = ("SELECT * FROM bookDB ORDER BY id ASC LIMIT 1;")
cur.execute(sql)
result = cur.fetchone()

print(result)

('1000034', '生死遗言', '伊能静', '现代出版社', None, None, '2002-10', 203, 1800, '元', '平装', '9787800288494', '伊能静，台湾女歌手、演员。音乐专辑代表作：《我是猫》、《悲伤的朱丽叶》、《落入凡间的精灵》；拍摄电影：《好男好女》、《南国·再见·南国》、《8 1/2的女人》等；电视剧：《七亿新娘》、《人间四月天》、《心灵物语》、《远嫁日本》。\n著有：《落入凡间的精灵》、《陪在你的左右》、《恋中的女人》、《为爱而生——伊能静自我写真》等书。\n', '《生死遗言》是伊能静的散文作品集，收有《生死遗言》、《消失的羽翼》、《迷雾森林》、《似水流年》、《有时跳舞》等篇。\n', '1 生死遗言\n2 静默\n3 你的信\n4 气息\n5 阴郁之心\n6 衣冠冢\n7 独角戏\n8 双面薇罗妮卡\n9 言语\n10 疼痛\n· · · · · ·     (\n)\n', '伊能静\n生死遗言\n随笔\n散文\n爱情\n台湾\n女性\n人生\n')


In [129]:
sql = "DROP TABLE shopDB"
cur.execute(sql)

In [68]:
create_userDB_sql = """
CREATE TABLE userDB (
    id SERIAL PRIMARY KEY,          -- 用户 ID，自增主键
    user_id TEXT NOT NULL ,
    password TEXT NOT NULL,         -- 密码
    token TEXT,                     -- Token
    terminal TEXT,                  -- 终端信息
    account INTEGER NOT NULL,          -- 账户余额
    address TEXT,                   -- 地址
    shops JSONB,                     -- 多个 shop.id
    orders JSONB 
    );
"""

cur.execute(create_userDB_sql)

In [70]:
sql = """
INSERT INTO userDB (user_id, password, token, terminal, account, address, shops, orders)
VALUES
(
    '张三',
    '123456',                                -- 密码
    'some_token',                                   -- Token
    'desktop',                                      -- 终端
     300,                                 -- 账户
    '123 User Address, City',                       -- 地址
    '[1, 2, 3]'::jsonb,                             -- shop.id 列表
    '[{"book_id": 101, "store_id": 5, "num": 2, "order_id": 5001, "state": "pending", "address": "123 Main St, City", "order_time": "20440344:2131"},
      {"book_id": 102, "store_id": 6, "num": 1, "order_id": 5002, "state": "shipped", "address": "456 Oak St, City", "order_time": "20440344:2131"}]'::jsonb  -- 订单列表，JSONB 格式
);
"""

cur.execute(sql)

In [71]:
sql = ("SELECT * FROM userDB ORDER BY id ASC LIMIT 1;")
cur.execute(sql)
result = cur.fetchone()

print(result)

(1, '张三', '123456', 'some_token', 'desktop', 300, '123 User Address, City', [1, 2, 3], [{'num': 2, 'state': 'pending', 'address': '123 Main St, City', 'book_id': 101, 'order_id': 5001, 'store_id': 5, 'order_time': '20440344:2131'}, {'num': 1, 'state': 'shipped', 'address': '456 Oak St, City', 'book_id': 102, 'order_id': 5002, 'store_id': 6, 'order_time': '20440344:2131'}])


In [24]:
con.commit()

In [130]:
create_shopDB_sql = """
CREATE TABLE shopDB (
    id SERIAL PRIMARY KEY,          -- 用户 ID，自增主键
    shop_id TEXT NOT NULL ,
    owner_id TEXT NOT NULL ,
    password TEXT NOT NULL ,
    books JSONB,
    orders JSONB                    -- 多个 order 字典，使用 JSONB 存储对象数组
);
"""
cur.execute(create_shopDB_sql)

In [27]:
sql = """
INSERT INTO shopDB (shop_id, owner_id, password, books, orders)
VALUES
(
    '淘宝',
    '刘大',
    'password',                                -- 密码
    '[
        {
            "book_id": "34144636",
            "num": 20,
            "price": 45,
            "book_info": "good"
        },
        {
            "book_id": "3242475",
            "num": 50,
            "price": 55,
            "book_info": "good"
        }
    ]'::jsonb,
    '[{
            "order_id": "fasge23445df",
            "buyer_id": "id1",
            "book_id": "345235235",
            "count": 1,
            "address": "shanghai",
            "state": "pending",
            "order_time": "2024-10-31T19:44:35.264736"
        },
        {
            "order_id": "daffwer123145",
            "buyer_id": "id2",
            "book_id": "345235237",
            "count": 1,
            "address": "shanghai",
            "state": "shipped",
            "order_time": "2024-10-31T19:44:35.264731"
        }]'::jsonb  -- 订单列表，JSONB 格式
);
"""

cur.execute(sql)

In [133]:
sql = ("SELECT * FROM shopDB ORDER BY id ASC LIMIT 1;")
cur.execute(sql)
result = cur.fetchone()

print(result)

con.commit()

(1, '天猫', '李四', 'password', [{'num': 20, 'price': 45, 'book_id': '34144636', 'book_info': 'good'}, {'num': 50, 'price': 55, 'book_id': '3242475', 'book_info': 'good'}], [{'count': 1, 'state': 'pending', 'address': 'shanghai', 'book_id': '345235235', 'buyer_id': 'id1', 'order_id': 'fasge23445df', 'order_time': '2024-10-31T19:44:35.264736'}, {'count': 1, 'state': 'shipped', 'address': 'shanghai', 'book_id': '345235237', 'buyer_id': 'id2', 'order_id': 'daffwer123145', 'order_time': '2024-10-31T19:44:35.264731'}])


In [52]:
sql = """
    UPDATE shopDB
    SET books = (
        SELECT jsonb_agg(
            CASE
                WHEN book->>'book_id' = %s THEN
                    jsonb_set(book, '{num}', ( (book->>'num')::int - %s )::text::jsonb)
                ELSE
                    book
            END
        )
        FROM jsonb_array_elements(books) AS book
    )
    WHERE shop_id = %s;
""",()

# 使用参数化查询，传入值为元组
cur.execute(sql, ('34144636', 1, '天猫'))


In [56]:
import json
new_order = {
    "order_id": "newOrder123",
    "buyer_id": "id3",
    "book_id": "345235238",
    "count": 2,
    "address": "beijing",
    "state": "pending",
    "order_time": "2024-12-25T00:00:00"
}

sql = """
UPDATE shopDB
SET orders = orders || %s::jsonb
WHERE shop_id = %s ;
"""
cur.execute(sql, [json.dumps([new_order]), '天猫', ])


In [57]:
sql = """
SELECT * FROM shopDB where shop_id = %s;
"""
params = ("天猫",)
cur.execute(sql, params)
result = cur.fetchone()
print(result)
if result is None:
    print("hh")
else:
    print(1)

(1, '天猫', '李四', 'password', [{'num': 19, 'price': 45, 'book_id': '34144636'}, None], [{'count': 1, 'state': 'pending', 'address': 'shanghai', 'book_id': '345235235', 'buyer_id': 'id1', 'order_id': 'fasge23445df', 'order_time': '2024-10-31T19:44:35.264736'}, {'count': 1, 'state': 'shipped', 'address': 'shanghai', 'book_id': '345235237', 'buyer_id': 'id2', 'order_id': 'daffwer123145', 'order_time': '2024-10-31T19:44:35.264731'}, {'count': 2, 'state': 'pending', 'address': 'beijing', 'book_id': '345235238', 'buyer_id': 'id3', 'order_id': 'newOrder123', 'order_time': '2024-12-25T00:00:00'}])
1


In [38]:
cur.execute("""
                    SELECT book->>'num' AS num, 
                           book->>'price' AS price
                    FROM shopDB,
                         jsonb_array_elements(books) AS book
                    WHERE shop_id = %s
                    AND book->>'book_id' = %s;""",
                ("天猫", "3242475"))

result = cur.fetchone()
print(result)
print(result[0])


('50', '55')
50


In [139]:
sql = """
SELECT * FROM shopDB where shop_id = %s;
"""

cur.execute(sql, ("天猫",))
result = cur.fetchone()
print(result)

(1, '天猫', '李四', 'password', [{'num': 20, 'price': 45, 'book_id': '34144636', 'book_info': 'good'}, {'num': 50, 'price': 55, 'book_id': '3242475', 'book_info': 'good'}, {'num': 9, 'price': 50, 'book_id': 'book_id', 'book_info': 'haha'}], [{'count': 1, 'state': 'pending', 'address': 'shanghai', 'book_id': '345235235', 'buyer_id': 'id1', 'order_id': 'fasge23445df', 'order_time': '2024-10-31T19:44:35.264736'}, {'count': 1, 'state': 'shipped', 'address': 'shanghai', 'book_id': '345235237', 'buyer_id': 'id2', 'order_id': 'daffwer123145', 'order_time': '2024-10-31T19:44:35.264731'}])


In [124]:
# 获取指定order
sql = """
SELECT order_item
FROM shopDB,
jsonb_array_elements(orders) AS order_item
WHERE order_item->>'order_id' = %s AND shop_id = %s;
"""
order_id = "newOrder123"
shop_id = "天猫"
cur.execute(sql, (order_id, shop_id))
result = cur.fetchall()
print(result)

[({'count': 3, 'state': 'paid1', 'address': 'beijing', 'book_id': '345235238', 'buyer_id': 'id3', 'order_id': 'newOrder123', 'order_time': '2024-12-25T00:00:00'},)]


In [123]:
sql = """
    UPDATE shopDB
SET orders = (
    SELECT jsonb_agg(
        CASE
            WHEN "order"->>'order_id' = %s THEN
                jsonb_set("order", '{count}', '3')
                
            ELSE
                "order"
        END
    )
    FROM jsonb_array_elements(orders) AS "order"
)
WHERE shop_id = %s;
"""

cur.execute(sql, ("newOrder123", "天猫"))


In [127]:
num = 5
sql = """
UPDATE userDB
SET account = account - %s
WHERE user_id = %s;
"""
cur.execute(sql, (num, "张三",))

In [149]:
sql = """
SELECT * FROM userDB where user_id = %s;
"""

cur.execute(sql, ("张三",))
result = cur.fetchone()
print(result)

(1, '张三', '123456', '1111', '222', 195, '123 User Address, City', [1, 2, 3], [{'num': 2, 'state': 'pending', 'address': '123 Main St, City', 'book_id': 101, 'order_id': 5001, 'store_id': 5, 'order_time': '20440344:2131'}, {'num': 1, 'state': 'shipped', 'address': '456 Oak St, City', 'book_id': 102, 'order_id': 5002, 'store_id': 6, 'order_time': '20440344:2131'}])


In [134]:
book_data = {
                'pages': None,
                'price': None,}
book_dict = {
    'pages': 5,
    'price':30
}
for key in book_data.keys():
    if key in book_dict:
        book_data[key] = book_dict[key]
        if key == "price":
            price = book_data[key]
print(price)

30


In [136]:
new_book = {
                "book_id": "book_id",
                "num" : 4,
                "price" : 50,
                "book_info" : "haha"
            }
cur.execute(
    """
    UPDATE shopDB
    SET books = books || %s::jsonb
    WHERE shop_id = %s ;
    """,
    [json.dumps([new_book]), "天猫"]
)

In [138]:
sql = """
UPDATE shopDB
                    SET books = (
                        SELECT jsonb_agg(
                            CASE
                                WHEN book->>'book_id' = %s THEN
                                    jsonb_set(book, '{num}', ( (book->>'num')::int + %s )::text::jsonb)
                                ELSE
                                    book
                            END
                        )
                        FROM jsonb_array_elements(books) AS book
                    )
                    WHERE shop_id = %s;
"""
cur.execute(sql, ("book_id", 5, "天猫"))

In [144]:
store_id = "淘宝"
user_id = "yzn"
password = "123456"
books = None
orders = None
sql = """
            INSERT INTO shopDB (shop_id, owner_id, password, books, orders)
            VALUES (%s, %s, %s, %s, %s);
            """
cur.execute(sql, (store_id, user_id, password, books, orders))

In [26]:
sql = """
SELECT * FROM shopDB;
"""

cur.execute(sql)
result = cur.fetchall()
print(result)

[(1, '天猫', '李四', 'password', [{'num': 20, 'price': 45, 'book_id': '34144636', 'book_info': 'good'}, {'num': 50, 'price': 55, 'book_id': '3242475', 'book_info': 'good'}, {'num': 9, 'price': 50, 'book_id': 'book_id', 'book_info': 'haha'}], [{'count': 1, 'state': 'shipped', 'address': 'shanghai', 'book_id': '345235237', 'buyer_id': 'id2', 'order_id': 'daffwer123145', 'order_time': '2024-10-31T19:44:35.264731'}])]


In [64]:
sql = """SELECT order_item
            FROM shopDB,
            jsonb_array_elements(orders) AS order_item
            WHERE shop_id = %s;"""
cur.execute(sql,("天猫",))
row = cur.fetchall()
print(row)
for r in row:
    print(r[0]["count"])

[({'count': 1, 'state': 'shipped', 'address': 'shanghai', 'book_id': '345235237', 'buyer_id': 'id2', 'order_id': 'daffwer123145', 'order_time': '2024-10-31T19:44:35.264731'},)]
1


In [22]:
cur.execute("""
            SELECT 
    order_item
FROM 
    shopDB,
    jsonb_array_elements(orders) AS order_item
WHERE 
    shop_id = '天猫' AND
    order_item->>'state' != 'received';
            """, ("天猫",))
row = list(cur.fetchall())
print(row)

[({'count': 1, 'state': 'paid3', 'address': 'shanghai', 'book_id': '345235235', 'buyer_id': 'id1', 'order_id': 'fasge23445df', 'order_time': '2024-10-31T19:44:35.264736'},), ({'count': 1, 'state': 'shipped', 'address': 'shanghai', 'book_id': '345235237', 'buyer_id': 'id2', 'order_id': 'daffwer123145', 'order_time': '2024-10-31T19:44:35.264731'},)]


In [60]:
sql = """
UPDATE shopDB
SET orders = (
SELECT jsonb_agg("order")
FROM jsonb_array_elements(orders) AS "order"
)
WHERE shop_id = %s; 
"""
cur.execute(sql, ("天猫",))
print(cur.fetchall())

ProgrammingError: no results to fetch

In [39]:
sql = """
SELECT 
    shop_id,
    "order"->>'order_id' AS order_id,
    "order"->>'order_time' AS order_time,
    "order"->>'buyer_id' AS buyer_id
FROM 
    shopDB,
    jsonb_array_elements(orders) AS "order"
WHERE 
    "order"->>'state' != 'ordered';
"""
cur.execute(sql)
store_data = cur.fetchall()
for store_item in store_data:
    for item in store_item:
        if item is not None:
            print("j")

j
j
j
j
j
j
j
j
j
j
j
j


In [50]:
from psycopg2 import sql
# 40%
def search_books(keyword: str, store_id: str):
    page = 1
    per_page = 10
    # 创建全文搜索查询，匹配 title, book_intro, author, content, author_intro 字段
    search_query = sql.SQL(
        "to_tsvector('english', title) || ' ' || "
        "to_tsvector('english', book_intro) || ' ' || "
        "to_tsvector('english', author) || ' ' || "
        "to_tsvector('english', content) || ' ' || "
        "to_tsvector('english', author_intro)"
    )

    # 使用 sql.Literal 来处理 keyword 参数
    ts_query = sql.SQL("to_tsquery('english', {})").format(sql.Literal(keyword))

    # 查询的基础部分
    query = sql.SQL("""
            SELECT id, title, book_intro, author, content, author_intro
            FROM bookDB
            WHERE {} @@ {}
        """).format(search_query, ts_query)
    
    # 分页
    offset = (page - 1) * per_page
    # 如果提供了 store_id，增加对商店中书籍ID的过滤
    if store_id:
        query += sql.SQL("""
                AND book_id IN (
                    SELECT unnest(books)
                    FROM shopDB
                    WHERE shop_id = %s
                    LIMIT %s OFFSET %s
                )
            """)
        # 执行查询时，传递正确的参数
        cur.execute(query, (store_id, per_page, offset))
    else:
        query += sql.SQL("""
            LIMIT %s OFFSET %s
        """)
        # 执行查询时，传递正确的参数
        cur.execute(query, (per_page, offset))

    results = cur.fetchall()
    print("Books:", results)

    # 获取总数查询
    total_query = sql.SQL("""
                SELECT COUNT(*)
                FROM bookDB
                WHERE {} @@ {}
            """).format(search_query, ts_query)

    # 如果 store_id 存在，添加商店过滤
    if store_id:
        total_query += sql.SQL("""
                    AND book_id IN (
                        SELECT unnest(books)
                        FROM shopDB
                        WHERE shop_id = %s
                    )
                """)
        # 执行总数查询时，传递正确的参数
        cur.execute(total_query, (store_id,))
    else:
        # 执行总数查询时，不需要 store_id 参数
        cur.execute(total_query)

    total_count = cur.fetchone()[0]  # 获取总数
    print("Total count:", total_count)
    
    return True

In [51]:
keyword = "伊能静"
store_id = None
search_books(keyword, store_id)

Books: [('27082517', '我们为什么被霸凌？', '400天田野调查，15个触目惊心的故事，解开校园霸凌的暗黑魔方！武志红、伊能静、李雪、凯叔、樊登、佟丽华、李国庆阅后联合力荐！\n《我们为什么被霸凌？》经过400天田野调查，呈现了15个触目惊心的校园霸凌故事，横跨小学、初中、高中、大学四个阶段，涵盖武力霸凌、流言霸凌、性 别霸凌、群体霸凌、校园霸凌等多种形式，揭示了99%的霸凌事件都可归因于家庭教育！\n作家陈岚在《我们为什么被霸凌？》中用写实的笔触，将你遭遇过的、没遭遇过的、想得到的、想不到的霸凌事件做了全面且深刻的解剖，并为学校、家长和孩子提出了具体可行的解决方案。多数派眼里，不合群的少数派永远是可爱的猎物！霸凌者眼里，无作为的监护人永远是得力的帮凶！\n穿着太土气？相貌太平庸？性格太软弱？成绩太优秀？\n探明源头，才能真正将霸凌扼杀在摇篮里！\n◎400天田野调查，15个触目惊心的故事，解开校园霸凌的暗...\n', '陈岚', '霸凌——从我的原创伤出发\n以此开始  小孩子的残酷\n第一幕  不是所有小孩都天真无邪\n哈洛的猴子\n校园寂静岭\n忘川之水\n第二幕  躁动的青春不止荷尔蒙\n小纰漏\n凶器是一本书\n黑武士\n白武士\n第三幕  作恶的世界里没有法不责众\n以伐木累之名\n同学会\n我不杀伯仁\n铁背心\n第四幕  嫉妒是骨中的朽烂\n朱鹮的安魂曲\n疯人院奇缘\n尚未结束  投胎是个技术活\n· · · · · ·     (\n)\n', '陈岚\n作家，主持人，儿童公益组织“小希望公益联盟”创办者，先后创立多家青少年保护机构，很多孩子的妈妈，女性及儿童心理健康发展研究者。\n'), ('1000034', '生死遗言', '《生死遗言》是伊能静的散文作品集，收有《生死遗言》、《消失的羽翼》、《迷雾森林》、《似水流年》、《有时跳舞》等篇。\n', '伊能静', '1 生死遗言\n2 静默\n3 你的信\n4 气息\n5 阴郁之心\n6 衣冠冢\n7 独角戏\n8 双面薇罗妮卡\n9 言语\n10 疼痛\n· · · · · ·     (\n)\n', '伊能静，台湾女歌手、演员。音乐专辑代表作：《我是猫》、《悲伤的朱丽叶》、《落入凡间的精灵》；拍摄电影：《好男好女》、《南国·再见·南国》、《8 1/2的女人》等；电

True

In [55]:
sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'shopdb' AND column_name = 'books';
"""

cur.execute(sql)
print(cur.fetchall())

[('books', 'jsonb')]


In [56]:
sql = """
select * from shopdb where shop_id = %s;
"""
cur.execute(sql, ("test_auto_cancel_order_dd417f37-c372-11ef-8ff6-900f0cb4fb46",))
print(cur.fetchone())

(478, 'test_auto_cancel_order_dd417f37-c372-11ef-8ff6-900f0cb4fb46', 'test_auto_cancel_order_dd417f36-c372-11ef-bab3-900f0cb4fb46', 'test_auto_cancel_order_dd417f36-c372-11ef-bab3-900f0cb4fb46', None, None)


In [59]:
cur.execute("SELECT books FROM shopdb WHERE shop_id = %s;", ("天猫",))
result = cur.fetchone()

# 假设 `result[0]` 是 `books` 字段的内容
books = result[0]

# 尝试解析并判断类型
if isinstance(books, dict):
    print("books 是一个 JSON 对象")
elif isinstance(books, list):
    print("books 是一个 JSON 数组")
else:
    print("books 既不是对象也不是数组")

books 是一个 JSON 数组
